# M1.1 Understanding Vector Databases
### Vector Search Foundations for RAG

**Learning Objectives:**
- Understand the semantic gap and why vector databases solve it
- Learn how embeddings represent meaning as 1536-dimensional vectors
- Master cosine similarity calculations
- Create and query Pinecone indexes with proper error handling
- Apply metadata filtering and score thresholding
- Debug common failures and prevent them

**Duration:** 60-90 minutes

---

## Section 1: Why Vector Databases?

### The Semantic Gap Problem

Traditional keyword search fails to understand **meaning**:
- Query: "climate change impacts"
- Misses: "global warming effects", "environmental consequences of CO2 emissions"

Vector databases solve this through **semantic search** - understanding intent, not just matching keywords.

### Vector Embeddings: Meaning as Numbers

Embeddings convert text → high-dimensional numerical vectors:
- OpenAI's `text-embedding-3-small`: **1536 dimensions**
- Similar meanings = close together in vector space
- Different meanings = far apart

Let's see this in action:

In [ ]:
# Generate embeddings for similar and different sentences
from openai import OpenAI
import config

# Initialize client
client = OpenAI(api_key=config.OPENAI_API_KEY)

# Three sentences: two similar, one different
sentences = [
    "The weather is beautiful today",
    "It's a gorgeous sunny day",
    "Python is a programming language"
]

# Generate embeddings
embeddings = []
for sentence in sentences:
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=sentence
    )
    embedding = response.data[0].embedding
    embeddings.append(embedding)
    print(f"Sentence: {sentence}")
    print(f"Dimensions: {len(embedding)}")
    print(f"First 5 values: {embedding[:5]}")
    print()

# Expected output:
# Dimensions: 1536
# Each sentence has unique 1536-dimensional vector

### Cosine Similarity: Measuring Semantic Distance

To find similar meanings, we calculate **cosine similarity** between vectors:
- **1.0** = identical meaning
- **0.0** = unrelated
- **-1.0** = opposite meaning

Formula: `similarity = (A · B) / (||A|| × ||B||)`

In [ ]:
# Calculate cosine similarity
import numpy as np

def cosine_similarity(vec1, vec2):
    """Calculate cosine similarity between two vectors"""
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    return dot_product / (norm1 * norm2)

# Compare sentence 0 with others
print("Similarity Scores:")
print(f"Sentence 0 vs 1 (both about weather): {cosine_similarity(embeddings[0], embeddings[1]):.4f}")
print(f"Sentence 0 vs 2 (weather vs programming): {cosine_similarity(embeddings[0], embeddings[2]):.4f}")

# Expected output:
# Sentence 0 vs 1: ~0.85 (high similarity - both about weather)
# Sentence 0 vs 2: ~0.12 (low similarity - different topics)

**Key Insight:** The weather sentences have high similarity (~0.85) despite different words. The programming sentence has low similarity (~0.12). This is semantic search in action!

---

## Section 2: Setting Up

### Prerequisites

**Required:**
1. Python 3.8+
2. OpenAI API key (for embeddings)
3. Pinecone API key (for vector database)

**Installation:**
```bash
pip install -r requirements.txt
```

**Dependencies:**
- `openai==1.46.0` - Embedding generation
- `pinecone-client==5.0.1` - Vector database
- `numpy==1.26.4` - Similarity calculations
- `python-dotenv==1.0.1` - Environment config
- `chromadb==0.5.5` - Alternative comparison (optional)

### Environment Configuration

**Step 1: Create `.env` file**
```bash
cp .env.example .env
```

**Step 2: Add your API keys**
```env
OPENAI_API_KEY=sk-proj-...
PINECONE_API_KEY=pcsk_...
PINECONE_REGION=us-east-1
```

**Get API keys:**
- OpenAI: https://platform.openai.com/api-keys
- Pinecone: https://app.pinecone.io/ (free tier available)

In [ ]:
# Verify configuration
import config

print("Configuration Validation")
print("=" * 50)
config.validate_config()
print("=" * 50)

# Expected output:
# ✓ OPENAI_API_KEY is set
# ✓ PINECONE_API_KEY is set
# ✓ PINECONE_REGION: us-east-1
# ✓ EMBEDDING_MODEL: text-embedding-3-small
# ✓ EMBEDDING_DIM: 1536
# ✓ INDEX_NAME: tvh-m1-vectors

In [ ]:
# Initialize clients
from openai import OpenAI
from pinecone import Pinecone

openai_client, pinecone_client = config.get_clients()

print("✓ OpenAI client initialized")
print("✓ Pinecone client initialized")
print("\nReady to create vector database!")

### Understanding the Configuration

**Key Constants (from `config.py`):**

```python
EMBEDDING_MODEL = "text-embedding-3-small"  # 1536 dimensions
EMBEDDING_DIM = 1536                         # MUST match model!
INDEX_NAME = "tvh-m1-vectors"               # Your index name
DEFAULT_NAMESPACE = "demo"                   # Data partition
SCORE_THRESHOLD = 0.7                        # Minimum similarity
BATCH_SIZE = 100                             # Vectors per batch
```

**Critical:** `EMBEDDING_DIM` must match your model:
- `text-embedding-3-small` → 1536 dimensions
- `text-embedding-3-large` → 3072 dimensions

Mismatch = **dimension error** (we'll debug this in Section 6).

---

## Section 3: Pinecone Basics

### Creating an Index

A Pinecone **index** is like a database table for vectors. You must specify:
- **Dimension**: Must match your embedding model (1536 for text-embedding-3-small)
- **Metric**: Distance calculation method (cosine, euclidean, or dotproduct)
- **Spec**: Deployment type (serverless or pod-based)

**Important:** Indexes take 30-60 seconds to initialize. We'll wait for readiness.

In [ ]:
# Create Pinecone index with readiness polling
from m1_1_vector_databases import create_index_and_wait_pinecone

index = create_index_and_wait_pinecone(
    pinecone_client,
    index_name=config.INDEX_NAME,
    dimension=config.EMBEDDING_DIM,
    metric=config.METRIC
)

print(f"✓ Index '{config.INDEX_NAME}' is ready!")
print(f"\nIndex stats:")
print(index.describe_index_stats())

# Expected output:
# Creating index: tvh-m1-vectors
#   Dimension: 1536
#   Metric: cosine
#   Region: us-east-1
# Waiting for index initialization...
#   Index still initializing...
# ✓ Index ready after 42.3 seconds

### Namespaces: Organizing Your Data

**Namespaces** partition data within a single index. Use cases:
- Multi-tenancy (separate data per user/organization)
- Environment isolation (dev/staging/production)
- Domain separation (tech docs vs financial docs)

**Benefits:**
- Query isolation (User A can't see User B's data)
- Cost efficiency (one index, multiple tenants)
- Performance (filter before search)

We'll use the `demo` namespace for this tutorial.

---

## Section 4: Upserting Data

### Loading Example Data

Let's load our example documents (diverse topics for testing semantic search):

In [ ]:
# Load example texts
from m1_1_vector_databases import load_example_texts

texts = load_example_texts("example_data.txt")

print(f"Loaded {len(texts)} documents")
print("\nFirst 3 documents:")
for i, text in enumerate(texts[:3]):
    print(f"{i+1}. {text}")

# Expected output:
# Loaded 20 documents
# 1. Vector databases enable semantic search using embeddings...
# 2. Pinecone is a managed vector database designed...
# 3. Climate change is causing rising sea levels...

### Generating Embeddings with Rate Limit Handling

**Important:** OpenAI has rate limits:
- Free tier: ~3,000 requests/minute
- Paid tier: ~10,000 requests/minute

Our `embed_texts_openai` function includes **exponential backoff** to handle rate limits gracefully.

In [ ]:
# Generate embeddings with retry logic
from m1_1_vector_databases import embed_texts_openai
from datetime import datetime

print(f"Generating embeddings at {datetime.now().strftime('%H:%M:%S')}...")

embeddings = embed_texts_openai(
    texts,
    client=openai_client,
    model=config.EMBEDDING_MODEL,
    max_retries=3
)

print(f"\n✓ Generated {len(embeddings)} embeddings")
print(f"  Dimension: {len(embeddings[0])}")
print(f"  Model: {config.EMBEDDING_MODEL}")

# Expected output:
# Generating embeddings for 20 texts using text-embedding-3-small
# Embedding texts: 100%|██████████| 20/20 [00:03<00:00,  5.67it/s]
# ✓ Generated 20 embeddings (dimension: 1536)

### Upserting with Rich Metadata

**Critical:** Always store metadata with your vectors!

**Required metadata:**
- `text`: The original document content (for retrieval)

**Recommended metadata:**
- `source`: Where the document came from
- `chunk_id`: Position in original document
- `timestamp`: When it was indexed
- Custom fields for filtering (category, user_id, etc.)

**Why metadata matters:**
- Without `text`, you only get IDs and scores (useless!)
- Filtering enables multi-tenancy and domain restriction
- Debugging and auditing

In [ ]:
# Prepare vectors with rich metadata
from m1_1_vector_databases import upsert_vectors

vectors = []
for i, (text, embedding) in enumerate(zip(texts, embeddings)):
    vectors.append((
        f"doc_{i}",  # Unique ID
        embedding,   # 1536-D vector
        {
            "text": text,                            # Original content
            "source": "example_data.txt",           # Source file
            "chunk_id": i,                          # Position
            "timestamp": datetime.utcnow().isoformat(),  # When indexed
            "length": len(text)                     # Character count
        }
    ))

# Upsert to Pinecone (batched automatically)
stats = upsert_vectors(
    index,
    vectors,
    namespace=config.DEFAULT_NAMESPACE,
    batch_size=config.BATCH_SIZE
)

print("\n✓ Upsert complete!")
print(f"  Upserted: {stats['upserted']} vectors")
print(f"  Namespace: {stats['namespace']}")
print(f"  Total in namespace: {stats['total_in_namespace']}")

# Expected output:
# Upserting 20 vectors to namespace 'demo'
# Batch size: 100
#   Batch 1: Upserted 20/20 vectors
# ✓ Upsert complete: 20 vectors

### Cost & Latency Considerations

**Embedding Costs (OpenAI):**
- `text-embedding-3-small`: $0.02 per 1M tokens
- ~750 words = 1,000 tokens
- 20 documents (~3,000 words) ≈ $0.00008 (negligible)

**Pinecone Costs:**
- Free tier: 100K vectors, 1 pod (adequate for learning)
- Serverless: $70/month + $0.40 per 1M queries
- Pod-based: $200+/month for dedicated capacity

**Latency breakdown:**
- Embedding generation: 10-50ms per request
- Pinecone upsert: 30-80ms (batched)
- Pinecone query: 30-80ms
- **Total query pipeline: 50-150ms minimum**

**Production tips:**
- Batch embeddings (reduce API overhead)
- Use namespaces strategically (reduce query scope)
- Monitor usage in Pinecone console
- Consider caching for frequently queried embeddings

---

## Section 5: Querying & Filtering

### Basic Semantic Search

Let's query our vector database with natural language:

In [ ]:
# Query 1: Vector search basics
from m1_1_vector_databases import query_pinecone

query1 = "What is vector search?"

results = query_pinecone(
    index,
    query1,
    client=openai_client,
    top_k=3,
    namespace=config.DEFAULT_NAMESPACE,
    score_threshold=0.7
)

print(f"Query: '{query1}'\\n")
for i, result in enumerate(results, 1):
    print(f"{i}. Score: {result['score']:.4f}")
    print(f"   Text: {result['text'][:100]}...")
    print()

# Expected output:
# Query: 'What is vector search?'
#
# 1. Score: 0.8923
#    Text: Vector databases enable semantic search using embeddings...
#
# 2. Score: 0.8156
#    Text: Pinecone is a managed vector database designed...

In [ ]:
# Query 2: Different topic
query2 = "climate and environmental issues"

results2 = query_pinecone(
    index,
    query2,
    client=openai_client,
    top_k=3,
    score_threshold=0.7
)

print(f"Query: '{query2}'\\n")
for i, result in enumerate(results2, 1):
    print(f"{i}. Score: {result['score']:.4f}")
    print(f"   Text: {result['text'][:80]}...")
    print()

# Expected output shows climate-related documents with high scores

### Understanding Score Thresholds

**Similarity scores range from -1 to 1 (cosine similarity):**
- **0.9+**: Nearly identical meaning
- **0.7-0.9**: Highly similar (recommended threshold for production)
- **0.5-0.7**: Moderately similar
- **<0.5**: Weakly related or unrelated

**Why thresholds matter:**
- Including low-score results pollutes your RAG context
- LLM performance degrades with irrelevant information
- Balance precision (high threshold) vs recall (low threshold)

Let's demonstrate the impact of threshold:

In [ ]:
# Compare thresholds
query = "machine learning concepts"

print("Threshold Comparison:\\n")

for threshold in [0.5, 0.7, 0.9]:
    results = query_pinecone(
        index,
        query,
        client=openai_client,
        top_k=5,
        score_threshold=threshold
    )
    print(f"Threshold {threshold}: {len(results)} results")
    if results:
        print(f"  Top score: {results[0]['score']:.4f}")
    print()

# Expected output shows fewer results with higher thresholds
# Threshold 0.5: 5 results (some may be irrelevant)
# Threshold 0.7: 3 results (good quality)
# Threshold 0.9: 1 result (very strict)

---

## Section 6: Debugging & Common Failures

This section covers the 5 most common errors you'll encounter and how to fix them.

### Failure #1: Dimension Mismatch

**Symptom:**
```
PineconeException: Vector dimension 3072 does not match index dimension 1536
```

**Cause:** Using wrong embedding model for your index dimension.

**Demo (will fail intentionally):**

In [ ]:
# DON'T RUN THIS - it will error!
# Demonstrating dimension mismatch

# Wrong model (3072-D) for 1536-D index
# response = client.embeddings.create(
#     model="text-embedding-3-large",  # 3072 dimensions!
#     input="test"
# )
# index.upsert([{"id": "test", "values": response.data[0].embedding}])
# → ERROR: Dimension mismatch!

print("✓ Skipped dimension mismatch demo (would error)")
print("\\nFix: Always match model dimension to index dimension")
print("text-embedding-3-small → 1536-D index")
print("text-embedding-3-large → 3072-D index")

### Failure #2: Missing Metadata

**Symptom:**
```python
KeyError: 'text'  # or empty metadata: {}
```

**Cause:** Stored vectors without metadata, can't retrieve original text.

**Fix:** Always include `text` field in metadata:

In [ ]:
# Bad: No metadata
bad_vector = {
    "id": "bad_example",
    "values": [0.1] * 1536  # Just the vector, no context!
}

# Good: Rich metadata
good_vector = {
    "id": "good_example",
    "values": [0.1] * 1536,
    "metadata": {
        "text": "Always store the original text!",  # REQUIRED
        "source": "best_practices.md",
        "chunk_id": 42,
        "category": "tutorial"
    }
}

print("Bad metadata causes retrieval failures!")
print("Good metadata enables:")
print("  ✓ LLM context generation")
print("  ✓ Source attribution")
print("  ✓ Metadata filtering")
print("  ✓ Debugging")

### Remaining Common Failures

**Failure #3: Ignoring Similarity Scores**
- **Problem:** Including all results regardless of score
- **Fix:** Apply threshold (0.7 recommended for production)
- **Detection:** Monitor score distribution in logs

**Failure #4: Rate Limit Exceeded**
- **Problem:** Hitting OpenAI rate limits (429 error)
- **Fix:** Implement exponential backoff (already in `embed_texts_openai`)
- **Prevention:** Batch requests, add delays between batches

**Failure #5: Index Not Ready**
- **Problem:** Querying immediately after index creation
- **Fix:** Use `create_index_and_wait_pinecone` (polls for readiness)
- **Prevention:** Always wait 30-60s for index initialization

**Debugging Checklist:**
- [ ] Embedding dimension matches index dimension
- [ ] Metadata includes 'text' field
- [ ] Score threshold is appropriate (0.7 default)
- [ ] Rate limiting is handled
- [ ] Index is ready before upserting/querying

---

## Bonus: ChromaDB Quick Comparison

ChromaDB is an **in-process** vector database - great for prototyping and small datasets.

**When to use ChromaDB:**
- Prototyping and local development
- Datasets < 1M vectors
- Cost-sensitive projects (free, open-source)
- Complete data control required

**When to use Pinecone:**
- Production workloads > 1M vectors
- Need managed infrastructure
- Multi-tenancy requirements
- Guaranteed 99.9% uptime SLA

**Quick ChromaDB demo:**

In [ ]:
# ChromaDB demo (optional - requires chromadb package)
import chromadb

# Create in-memory client
chroma_client = chromadb.Client()

# Create collection
collection = chroma_client.create_collection(name="demo_collection")

# Add documents (ChromaDB handles embeddings automatically!)
collection.add(
    documents=[
        "Vector databases enable semantic search",
        "ChromaDB is an open-source alternative",
        "Pinecone offers managed infrastructure"
    ],
    ids=["doc1", "doc2", "doc3"]
)

# Query
chroma_results = collection.query(
    query_texts=["What are vector database options?"],
    n_results=2
)

print("ChromaDB Results:")
for doc, distance in zip(chroma_results['documents'][0], chroma_results['distances'][0]):
    print(f"  - {doc} (distance: {distance:.4f})")

print("\\n✓ ChromaDB: Simple for prototyping, but lacks production features")
print("✓ Pinecone: Production-ready, managed, scalable")

---

## Summary & Next Steps

### What You Learned

✓ **Vector embeddings** convert text to 1536-D numerical representations  
✓ **Cosine similarity** measures semantic distance (-1 to 1)  
✓ **Pinecone indexes** store millions of vectors with ANN search  
✓ **Namespaces** enable multi-tenancy and data isolation  
✓ **Metadata** is critical for retrieval and filtering  
✓ **Score thresholds** (0.7 recommended) filter low-quality matches  
✓ **5 common failures** and how to debug them

### Decision Card: When to Use Vector Databases

**✅ Use when:**
- Semantic search is primary requirement
- Dataset > 10K documents
- Can accept 50-100ms latency overhead
- Need managed infrastructure

**❌ Avoid when:**
- Exact keyword matching required (legal/compliance)
- Dataset < 1K documents
- Real-time freshness < 1 second
- Budget constraints (use ChromaDB/pgvector)

### Production Checklist

Before deploying to production:
- [ ] Match embedding dimension to index dimension
- [ ] Store comprehensive metadata (especially 'text')
- [ ] Implement retry logic for rate limits
- [ ] Set appropriate score threshold (test on your data)
- [ ] Monitor costs in Pinecone console
- [ ] Use namespaces for multi-tenancy
- [ ] Batch operations (100-200 vectors)
- [ ] Wait for index readiness before operations

### Next Steps

1. **Practice:** Complete the CLI tool exercises in README.md
2. **Experiment:** Try different embedding models and thresholds
3. **Compare:** Test ChromaDB vs Pinecone on your use case
4. **Build:** Create a simple Q&A system with your own documents
5. **Monitor:** Track costs and latency in production

**Resources:**
- Pinecone docs: https://docs.pinecone.io/
- OpenAI embeddings: https://platform.openai.com/docs/guides/embeddings
- ChromaDB: https://docs.trychroma.com/
- Course repo: Check README.md for challenges

---

**Congratulations!** You now understand vector databases and are ready to build production RAG systems. 🎉